In [ ]:
import os
from improc.io import parse_collection, DCAccessor
from skimage.io import imread
DCAccessor.register()

import numpy as np
import holoviews as hv
hv.extension('bokeh')

# read example images

In [ ]:
img = imread('../../data/2D_corrections/raw/201102JD006AAB_201101_153213_B02_T0001F001L01A03Z01C01_roi_5243_8490_158_2336.tif')

# drawing interactive region of intereset (ROI)

The RoiEditor generates a hv.Rectangle plot than can be overlaid on any plots. It also converts the box location into python slices, taking in account data spacing.

To draw a roi, activate the box edit tool, press shift or double click and drag the mouse.

In [ ]:
from inter_view.view_images import ChannelViewer
from inter_view.edit_images import RoiEditor
from inter_view.utils import HvDataset
from holoviews import opts
import panel as pn

# show the axis 
opts.defaults(opts.Image('channel', xaxis='bottom', yaxis='left'))

roi_editor = RoiEditor(num_objects=1, spacing=(0.1,0.1))

hv_dataset = HvDataset(img=img, spacing=(0.1,0.1))
channel_viewer = ChannelViewer()
dmap = channel_viewer(hv_dataset.dmap())

pn.Row(dmap * roi_editor.roi_plot, pn.Row(roi_editor.img_slice))

## draw an ROI above before running

In [ ]:
# crop the original numpy array with the roi

crop_img = hv_dataset.img[roi_editor.img_slice()[0]]
crop_dataset = HvDataset(img=crop_img)
channel_viewer(crop_dataset.dmap())

# label editor

In [ ]:
from inter_view.view_images import ChannelViewer
from inter_view.edit_images import EditableHvDataset, FreehandEditor
from holoviews import opts
import panel as pn

opts.defaults(opts.Image(clipping_colors={'min': (0, 0, 0, 0)}, clim=(0,2**16-1), tools=['hover']))


img = np.zeros((256,256), dtype=np.int16)
img[:100,:100] = 1
img[150:200, 150:256] = 2

hv_edit_dataset = EditableHvDataset(img=img, spacing=(1,1))
freehand_editor = FreehandEditor(dataset=hv_edit_dataset)
channel_viewer = ChannelViewer(cmap='glasbey_hv_16bit',
                               intensity_bounds=(-2**15,2**15-1),
                               slider_limits=(-2**15,2**15-1),
                               raster_aggregator='first',
                               bitdepth='int16')

dmap = channel_viewer(hv_edit_dataset.dmap())

pn.Row(dmap * freehand_editor.path_plot, freehand_editor.widgets())

## edited image data can be recovered

In [ ]:
hv_edit_dataset.img